# Restricted Boltzmann Machines and Deep Belief Networks

## Restricted Boltzmann Machine

### Theory

A Restricted Boltzmann Machine (RBM) is a probabilistic graphical model that consists of visible random variables (or units) $\mathbf{v}$ and hidden random variables (or units) $\mathbf{h}$, where:

$$\mathbf{v}=\begin{bmatrix} v_1 \\ v_2 \\ \vdots \\ v_N \end{bmatrix}$$

And:

$$\mathbf{h}=\begin{bmatrix} h_1 \\ h_2 \\ \vdots \\ h_M \end{bmatrix}$$

See [section 16.5](https://www.deeplearningbook.org/contents/graphical_models.html) in the book titled "Deep Learning" by Ian Goodfellow et al. for why the hidden vector $\mathbf{h}$ is necessary.

The joint distribution represented by all [Boltzmann Machines](https://en.wikipedia.org/wiki/Boltzmann_machine) (not just an RBM) is:

$$p(\mathbf{v},\mathbf{h};\mathbf{W},\mathbf{a},\mathbf{b}) = \frac{1}{Z} e^{-E(\mathbf{v},\mathbf{h};\mathbf{W},\mathbf{a},\mathbf{b})}$$

Where $\mathbf{W}$ is a parameter matrix, $\mathbf{a}$ and $\mathbf{b}$ are parameter vectors, and:

$$E(\mathbf{v},\mathbf{h};\mathbf{W},\mathbf{a},\mathbf{b}) = -\mathbf{a}^T\mathbf{v} - \mathbf{b}^T\mathbf{h} - \mathbf{v}^T\mathbf{W}\mathbf{h}$$

$$Z = \sum_{\mathbf{v},\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{W},\mathbf{a},\mathbf{b})}$$

$E$ is know as the **energy function** and $Z$ is known as the **partition function**. However, to make parameter estimation (learning) and inference tractable, a **Restricted** Boltzmann Machine introduces the following assumptions:

$$ p(\mathbf{v}|\mathbf{h}) = \prod_{i=1}^N p(v_i|\mathbf{h})$$
$$p(\mathbf{h}|\mathbf{v}) = \prod_{j=1}^M p(h_i|\mathbf{v})$$

In words, it is assumed that the individual visible random variables (units) are conditionally independent given all the hidden units $\mathbf{h}$. Also, it is assumed that the individual hidden units are conditionally independent given all the visible units $\mathbf{v}$.

It is desirable to estimate the marginal probability distribution $p(\mathbf{v})$ of all the visible units $\mathbf{v}$, as this reveals relationships between the visible units. This marginal probability distribution can be obtained by marginalizing the joint distribution $p(\mathbf{v},\mathbf{h})$ over all the hidden units $\mathbf{h}$:

$$p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}) = \frac{1}{Z} \sum_{\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{W},\mathbf{a},\mathbf{b})}$$

This marginal probability distribution can therefore be estimated using maximum likelihood estimation. Assuming that the visible units $\mathbf{v}$ have been observed, then the equation above also represents the likelihood function. The log-likelihood function is therefore:

$$ln(p(\mathbf{v};\mathbf{\theta})) = ln\left(\sum_{\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}\right) - ln\left(\sum_{\mathbf{v},\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}\right)$$

Where:

$$\theta = \begin{bmatrix} \mathbf{W} \\ \mathbf{a} \\ \mathbf{b} \end{bmatrix}$$

For brevity.

Since the log-likelihood function is not in closed form, which means that it is difficult to compute analytically, then gradient ascent will be used to maximize it. The gradient of the log-likelihood function is:

$$\frac{\partial ln(p(\mathbf{v};\mathbf{\theta}))}{\partial \mathbf{\theta}} = -\frac{1}{\sum_{\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}} \sum_{\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}\right] +
\frac{1}{\sum_{\mathbf{v},\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}} \sum_{\mathbf{v},\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}\right]$$

Since:

$$p(\mathbf{h}|\mathbf{v}) = \frac{p(\mathbf{v},\mathbf{h})}{p(\mathbf{v})} = \frac{\frac{1}{Z} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}}{\frac{1}{Z} \sum_{\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}} = \frac{e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}}{\sum_{\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}}$$

Then:

$$\frac{\partial ln(p(\mathbf{v};\mathbf{\theta}))}{\partial \mathbf{\theta}} = -\sum_{\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot p(\mathbf{h}|\mathbf{v}) \right] +
\frac{1}{\sum_{\mathbf{v},\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}} \sum_{\mathbf{v},\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}\right]$$

Also, since:

$$\frac{e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}}{Z} = \frac{e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}}{\sum_{\mathbf{v},\mathbf{h}} e^{-E(\mathbf{v},\mathbf{h};\mathbf{\theta})}} = p(\mathbf{v},\mathbf{h})$$

Then:

$$\begin{align}
\frac{\partial ln(p(\mathbf{v};\mathbf{\theta}))}{\partial \mathbf{\theta}} &= -\sum_{\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot p(\mathbf{h}|\mathbf{v}) \right] +
\sum_{\mathbf{v},\mathbf{h}} \left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \cdot p(\mathbf{v},\mathbf{h})\right] \\
&= -\mathbb{E}_{p(\mathbf{h}|\mathbf{v})}\left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \right] + \mathbb{E}_{p(\mathbf{v},\mathbf{h})}\left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \right]
\end{align}$$



Since both of these are expectations, they can be approximated using [Monte Carlo integration](http://people.duke.edu/~ccc14/sta-663-2019/notebook/S14D_Monte_Carlo_Integration.html#Intuition-behind-Monte-Carlo-integration):

$$\frac{\partial ln(p(\mathbf{v};\mathbf{\theta}))}{\partial \mathbf{\theta}} \approx -\frac{1}{N} \sum_{i = 1}^{N} \left[\frac{\partial E(\mathbf{v},\mathbf{h}_i;\mathbf{\theta})}{\partial \mathbf{\theta}} \right] +
\frac{1}{M} \sum_{j=1}^{M} \left[\frac{\partial E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{\theta})}{\partial \mathbf{\theta}} \right]$$

The first term can be computed because it is easy to sample from $p(\mathbf{h}|\mathbf{v})$. However, it is difficult to sample from $p(\mathbf{v},\mathbf{h})$ directly, but since it is easy to sample from $p(\mathbf{v}|\mathbf{h})$, then Gibbs sampling is used to sample from both $p(\mathbf{h}|\mathbf{v})$ and $p(\mathbf{v}|\mathbf{h})$ to approximate a sample from $p(\mathbf{v},\mathbf{h})$. Also, notice that:

$$\begin{align}
\frac{\partial ln(p(\mathbf{v};\mathbf{\theta}))}{\partial \mathbf{\theta}} &= -\mathbb{E}_{p(\mathbf{h}|\mathbf{v})}\left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \right] + \mathbb{E}_{p(\mathbf{v},\mathbf{h})}\left[\frac{\partial E(\mathbf{v},\mathbf{h};\mathbf{\theta})}{\partial \mathbf{\theta}} \right] \\
&\approx - \frac{1}{N} \sum_{i = 1}^{N} \left[\frac{\partial E(\mathbf{v},\mathbf{h}_i;\mathbf{\theta})}{\partial \mathbf{\theta}} \right] + \frac{1}{M} \sum_{j=1}^{M} \left[\frac{\partial E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{\theta})}{\partial \mathbf{\theta}} \right] \\
&\approx \frac{\partial}{\partial \mathbf{\theta}} \left(\frac{1}{M} \sum_{j=1}^{M} \left[E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{\theta}) \right] - \frac{1}{N} \sum_{i = 1}^{N} \left[E(\mathbf{v},\mathbf{h}_i;\mathbf{\theta}) \right] \right)
\end{align}$$

In words, this means that the gradient of the log-likelihood function can be approximated by the gradient of the difference of two sample means. Once the gradient of the log-likelihood function is estimated for a particular set of observed visible units $\mathbf{v}$, then the parameters $\mathbf{W},\mathbf{a},$ and $\mathbf{b}$ can be updated using the following gradient ascent update rules:

$$\mathbf{W}_{t+1} = \mathbf{W}_{t} + \epsilon \frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{W}}$$

$$\mathbf{a}_{t+1} = \mathbf{a}_{t} + \epsilon \frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{a}}$$

$$\mathbf{b}_{t+1} = \mathbf{b}_{t} + \epsilon \frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{b}}$$

Where $\epsilon$ is the learning rate and:

$$\frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{W}} \approx \frac{\partial}{\partial \mathbf{W}} \left(\frac{1}{M} \sum_{j=1}^{M} \left[E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{W},\mathbf{a},\mathbf{b}) \right] - \frac{1}{N} \sum_{i = 1}^{N} \left[E(\mathbf{v},\mathbf{h}_i;\mathbf{W},\mathbf{a},\mathbf{b}) \right] \right)$$

$$\frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{a}} \approx \frac{\partial}{\partial \mathbf{a}} \left(\frac{1}{M} \sum_{j=1}^{M} \left[E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{W},\mathbf{a},\mathbf{b}) \right] - \frac{1}{N} \sum_{i = 1}^{N} \left[E(\mathbf{v},\mathbf{h}_i;\mathbf{W},\mathbf{a},\mathbf{b}) \right] \right)$$

$$\frac{\partial ln(p(\mathbf{v};\mathbf{W},\mathbf{a},\mathbf{b}))}{\partial \mathbf{b}} \approx \frac{\partial}{\partial \mathbf{b}} \left(\frac{1}{M} \sum_{j=1}^{M} \left[E(\mathbf{v}_j,\mathbf{h}_j;\mathbf{W},\mathbf{a},\mathbf{b}) \right] - \frac{1}{N} \sum_{i = 1}^{N} \left[E(\mathbf{v},\mathbf{h}_i;\mathbf{W},\mathbf{a},\mathbf{b}) \right] \right)$$

### Implementation

This implementation of an RBM is inspired by the paper titled [Restricted Boltzmann Machines for Collaborative Filtering](https://www.cs.toronto.edu/~rsalakhu/papers/rbmcf.pdf) by Ruslan Salakhutdinov et al. More precisely, one of the RBM architectures that the authors implemented involved categorical (or softmax) visible units and Bernoulli (or binary) hidden units. Also, in the paper, the authors use data from the [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize) competition that consists of approximately 100 million user ratings. 480,000 users gave approximately 18,000 movies a rating from 1 to 5. Therefore, the main objective of this implementation is to use these user ratings as the categorical visible units with Bernoulli hidden units and estimate the corresponding parameters using maximum likelihood estimation. However, to keep the implementation simple, only 50 movies will be considered.

This RBM will be implemented using PyTorch.

Given 50 movies, this means that there are 50 visible categorical units. Suppose that there are 25 hidden Bernoulli units. Note that since user ratings range from 1 to 5, they will need to be expressed in one-hot encoding format (see [here](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) for an explanation). Therefore, suppose that 50 movie ratings by a single user are obtained in one-hot encoding format:

In [4]:
import torch

# create a categorical probability distribution for each of the 50 movie ratings. Note
# that these probability distributions will be helpful for validating results later on

v = torch.randn((50,5))
v = torch.nn.functional.softmax(v,dim = 1)
v

tensor([[0.1721, 0.2829, 0.0214, 0.0192, 0.5043],
        [0.0867, 0.4595, 0.1345, 0.1894, 0.1299],
        [0.2326, 0.3708, 0.1514, 0.1246, 0.1206],
        [0.2243, 0.1991, 0.2731, 0.2845, 0.0189],
        [0.0191, 0.1250, 0.5236, 0.2545, 0.0778],
        [0.4259, 0.1737, 0.1083, 0.1265, 0.1656],
        [0.5671, 0.0830, 0.2285, 0.0806, 0.0408],
        [0.1066, 0.0758, 0.1777, 0.3223, 0.3176],
        [0.3902, 0.0944, 0.1910, 0.2639, 0.0605],
        [0.1557, 0.0934, 0.3536, 0.1880, 0.2092],
        [0.0094, 0.0158, 0.0719, 0.2030, 0.6999],
        [0.0303, 0.0359, 0.2922, 0.5208, 0.1208],
        [0.5213, 0.0612, 0.1281, 0.0755, 0.2138],
        [0.2511, 0.0487, 0.3005, 0.3665, 0.0332],
        [0.0609, 0.0891, 0.0405, 0.1755, 0.6340],
        [0.1500, 0.2950, 0.1157, 0.1814, 0.2579],
        [0.0960, 0.2146, 0.4050, 0.0175, 0.2669],
        [0.1827, 0.4937, 0.2408, 0.0797, 0.0031],
        [0.2269, 0.0329, 0.0906, 0.3974, 0.2521],
        [0.1084, 0.0185, 0.5870, 0.0602, 0.2258],


In [5]:
# check that each row adds up to 1

v.sum(dim = 1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [6]:
# sample 50 one-hot encoded vectors from each categorical distribution

v = torch.distributions.one_hot_categorical.OneHotCategorical(probs = v).sample()
v

tensor([[0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0

Next, the parameters $\mathbf{W},\mathbf{a},$ and $\mathbf{b}$ need to be initialized:

In [8]:
num_visible = 50
num_hidden = 25
W = torch.randn(num_hidden,num_visible)
a = torch.randn(num_visible,1)
b = torch.randn(num_hidden,1)

In [9]:
W

tensor([[ 1.7279,  0.0470,  0.2512,  ..., -0.7241, -0.0427, -1.6977],
        [-0.4283,  0.4730,  0.3296,  ...,  0.1101,  0.4922, -0.4110],
        [-1.7327, -1.9367, -1.1274,  ...,  0.8164, -0.5042,  1.1784],
        ...,
        [ 1.0491, -1.0257, -0.9975,  ..., -0.5079, -2.1602,  0.5719],
        [ 0.9663, -0.3578, -0.4960,  ...,  0.3218,  0.9213,  0.6263],
        [ 0.7383,  0.5950,  0.8808,  ...,  0.0477,  0.9073, -1.1476]])

In [10]:
a

tensor([[ 0.5862],
        [ 1.5568],
        [-0.2033],
        [ 0.0202],
        [-0.5568],
        [ 2.2841],
        [ 0.1458],
        [ 0.9263],
        [-0.2177],
        [ 2.0682],
        [ 0.0493],
        [ 0.1205],
        [-0.3217],
        [-1.7253],
        [ 0.5958],
        [-0.3676],
        [-1.6230],
        [-0.2149],
        [-0.2773],
        [-1.1704],
        [ 0.6007],
        [-2.5750],
        [-0.4187],
        [-0.7106],
        [-0.7953],
        [ 0.6599],
        [-0.4423],
        [ 2.0682],
        [-1.0603],
        [ 0.3971],
        [ 1.3892],
        [-0.9456],
        [-0.4934],
        [ 0.7437],
        [-0.0402],
        [-0.9114],
        [ 0.0707],
        [ 1.1751],
        [ 1.9328],
        [-0.5940],
        [ 1.5585],
        [-1.9806],
        [-0.8305],
        [-0.1005],
        [-1.4645],
        [ 0.0216],
        [ 0.7611],
        [-1.1748],
        [ 0.6165],
        [-0.1728]])

In [11]:
b

tensor([[ 0.8113],
        [ 0.5568],
        [-0.4792],
        [ 0.4992],
        [-1.2462],
        [-0.5796],
        [ 0.2035],
        [ 0.2655],
        [ 1.4925],
        [-1.4995],
        [-0.3472],
        [-2.0273],
        [ 1.2186],
        [-1.4558],
        [-0.4875],
        [ 0.0735],
        [ 0.2867],
        [ 0.0129],
        [-0.5585],
        [ 0.2561],
        [ 0.6355],
        [ 0.6224],
        [-1.5228],
        [ 1.6203],
        [-0.7766]])